In [ ]:
# ==========================================
# Importação de bibliotecas
# ==========================================
import json
import os
import pickle
import numpy as np
import faiss
from openai import OpenAI
from dotenv import load_dotenv

# ==========================================
# Configuração do ambiente e cliente OpenAI
# ==========================================
load_dotenv()  # Carregar variáveis do arquivo .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # Inicializar cliente OpenAI

# ==========================================
# Carregar os chunks de texto
# ==========================================
input_file = os.getenv("PROCESSED_JSON")  # Caminho do JSON de chunks

with open(input_file, "r", encoding="utf-8") as f:
    chunks = json.load(f)

print(f"Total de chunks carregados: {len(chunks)}")
print("Exemplo de chunk:", chunks[0])

# ==========================================
# Gerar embeddings para cada chunk
# ==========================================
embeddings_list = []

for chunk in chunks:
    response = client.embeddings.create(
        model="text-embedding-3-large",
        input=chunk["text"]
    )
    
    emb = response.data[0].embedding
    
    embeddings_list.append({
        "chunk_number": chunk["chunk_number"],
        "embedding": emb,
        "source_file": chunk["source_file"],
        "page_number": chunk["page_number"]
    })

print("✅ Embeddings gerados para todos os chunks!")

# ==========================================
# Criar matriz de embeddings e indexar com FAISS
# ==========================================
X = np.array([item["embedding"] for item in embeddings_list]).astype("float32")

# Garantir que os embeddings na lista também estão como float32
for i, item in enumerate(embeddings_list):
    embeddings_list[i]["embedding"] = X[i]

# Criar índice FAISS usando distância L2 e adicionar embeddings
index = faiss.IndexFlatL2(X.shape[1])
index.add(X)

# ==========================================
# Salvar índice FAISS e metadados
# ==========================================

faiss.write_index(index, "embeddings.index")
with open("metadata.pkl", "wb") as f:
    pickle.dump(embeddings_list, f)

print(f"✅ Embeddings e metadados salvos")


Total de chunks carregados: 60
Exemplo de chunk: {'chunk_number': 0, 'text': 'CONTRATO DE ASSOCIAÇÃO DE EMPRESA Pelo presente instrumento e na melhor forma de direito, de um lado, AJJ SERVIÇOS CONTABÉIS LTDA, pessoa jurídica de direito privado, inscrita no CNPJ sob o nº 28.913.706/0001-63, com sede na Rua Alfred Charvet, nº 710, Bairro Vila Nova, Araucária/PR, CEP: 83.703-278, neste ato, representada por seu sócio administrador, Jhonny Cézar de Jesus Falavinha, brasileiro, portador do RG nº 6.223.402-4 SSP/PR, inscrito no CPF nº 007.724.809-07, doravante denominada EMPRESA ASSOCIADA, e de outro lado, o PARQUE CIENTÍFICO E TECNOLÓGICO DE BIOCIÊNCIAS LTDA, pessoa jurídica de direito privado, inscrito no CNPJ sob o nº 21.526.709/0001-03, com sede na Rodovia PR-182, Km 320/321, s/n, Condomínio Industrial Biopark, no município de Toledo, Estado do Paraná, CEP 85.919-899, neste ato representada por seu sócio administrador, Victor Donaduzzi, brasileiro, inscrito no CPF sob o nº 047.139.439-40